## **Big Data Analytics Project**

## **Developed by :**

# Abu Bakar Siddiq

# **Large-Scale Flight Delay Analysis Using Apache Spark and PySpark**

**Install Java & Spark**

In [ ]:
!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!tar -xvf spark-3.5.0-bin-hadoop3.tgz

spark-3.5.0-bin-hadoop3/
spark-3.5.0-bin-hadoop3/kubernetes/
spark-3.5.0-bin-hadoop3/kubernetes/tests/
spark-3.5.0-bin-hadoop3/kubernetes/tests/pyfiles.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning_cleanup.py
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/decom.sh
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/python/
spark-3.5.0-bin-hadoop3/kubern

**Set Environment Variables**

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

**Start Spark Session**

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Flight Delay Big Data Analysis") \
    .config("spark.ui.showConsoleProgress", "true") \
    .getOrCreate()

spark

**Loading Dataset into Spark & Displaying Schema**

In [ ]:
df = spark.read.csv(
    "flights.csv",
    header=True,
    inferSchema=True
)

df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

**Convert Arrival_Delay to numeric & remove nulls**

In [ ]:
from pyspark.sql.functions import col

df_clean = df.select(
    col("FLIGHT_NUMBER"),
    col("ARRIVAL_DELAY").cast("double")
).filter(col("ARRIVAL_DELAY").isNotNull())

df_clean.show(5)

+-------------+-------------+
|FLIGHT_NUMBER|ARRIVAL_DELAY|
+-------------+-------------+
|           98|        -22.0|
|         2336|         -9.0|
|          840|          5.0|
|          258|         -9.0|
|          135|        -21.0|
+-------------+-------------+
only showing top 5 rows


**CONVERT TO RDD**

In [ ]:
rdd = df_clean.rdd

**Applying Map**

In [ ]:
# MAP: (flight_number, arrival_delay)
mapped = rdd.map(lambda row: (row.FLIGHT_NUMBER, row.ARRIVAL_DELAY))

mapped.take(5)

[(98, -22.0), (2336, -9.0), (840, 5.0), (258, -9.0), (135, -21.0)]

**Applying Reduce**

In [ ]:
# REDUCE: average delay per flight
reduced = mapped.groupByKey() \
    .mapValues(lambda delays: sum(delays) / len(delays))

reduced.take(5)

[(98, -6.888888888888889),
 (2336, 5.111111111111111),
 (840, -1.9285714285714286),
 (258, 8.409090909090908),
 (806, 4.75)]

**SHOW SPARK INTERNALS**

In [ ]:
print("Number of partitions:", rdd.getNumPartitions())

Number of partitions: 2


In [ ]:
reduced.toDebugString()

b'(2) PythonRDD[41] at RDD at PythonRDD.scala:56 []\n |  MapPartitionsRDD[39] at mapPartitions at PythonRDD.scala:168 []\n |  ShuffledRDD[38] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(2) PairwiseRDD[37] at groupByKey at /tmp/ipython-input-798040233.py:2 []\n    |  PythonRDD[36] at groupByKey at /tmp/ipython-input-798040233.py:2 []\n    |  MapPartitionsRDD[34] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[33] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  SQLExecutionRDD[32] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[31] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[30] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  FileScanRDD[29] at javaToPython at NativeMethodAccessorImpl.java:0 []'

**FEATURE ENGINEERING (Spark ML)**

In [ ]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
# Select useful features for ML
ml_df = df.select(
    col("MONTH"),
    col("DAY"),
    col("AIRLINE"),
    col("ORIGIN_AIRPORT"),
    col("DESTINATION_AIRPORT"),
    col("DEPARTURE_DELAY"),
    col("ARRIVAL_DELAY")
).filter(col("ARRIVAL_DELAY").isNotNull()) \
 .filter(col("DEPARTURE_DELAY").isNotNull())

**ENCODE CATEGORICAL FEATURES**

In [ ]:
airline_indexer = StringIndexer(
    inputCol="AIRLINE",
    outputCol="AIRLINE_IDX",
    handleInvalid="skip"
)

origin_indexer = StringIndexer(
    inputCol="ORIGIN_AIRPORT",
    outputCol="ORIGIN_IDX",
    handleInvalid="skip"
)

dest_indexer = StringIndexer(
    inputCol="DESTINATION_AIRPORT",
    outputCol="DEST_IDX",
    handleInvalid="skip"
)

In [ ]:
ml_df = airline_indexer.fit(ml_df).transform(ml_df)
ml_df = origin_indexer.fit(ml_df).transform(ml_df)
ml_df = dest_indexer.fit(ml_df).transform(ml_df)

**VECTOR ASSEMBLER (Spark Internal Feature Vector)**

In [ ]:
assembler = VectorAssembler(
    inputCols=[
        "MONTH",
        "DAY",
        "DEPARTURE_DELAY",
        "AIRLINE_IDX",
        "ORIGIN_IDX",
        "DEST_IDX"
    ],
    outputCol="features"
)

final_ml_df = assembler.transform(ml_df).select("features", "ARRIVAL_DELAY")

**TRAIN–TEST SPLIT (Distributed)**

In [ ]:
train_df, test_df = final_ml_df.randomSplit([0.8, 0.2], seed=42)

**MACHINE LEARNING ALGORITHM Random Forest Regression (Spark ML)**

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="ARRIVAL_DELAY",
    numTrees=20,
    maxDepth=7,
    maxBins=400
)

In [ ]:
rf_model = rf.fit(train_df)

**PREDICTIONS**

In [ ]:
predictions = rf_model.transform(test_df)

predictions.select(
    "ARRIVAL_DELAY",
    "prediction"
).show(10)

+-------------+-------------------+
|ARRIVAL_DELAY|         prediction|
+-------------+-------------------+
|          -23|-1.6251738679655783|
|            6|  1.493188241556867|
|          -13|  -6.08350858733149|
|          -22| -7.587746626849937|
|          -21| -5.030351592643466|
|            0| -3.364195229730317|
|          -11| -8.407955117801906|
|          -18| -7.930384200847257|
|          -10|-10.041209007000212|
|          -11| -8.139554340983992|
+-------------+-------------------+
only showing top 10 rows


**MODEL EVALUATION (Big Data Scale)**

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="ARRIVAL_DELAY",
    predictionCol="prediction",
    metricName="r2"
)

r2 = r2_evaluator.evaluate(predictions)

print("RMSE:", rmse)
print("R2 Score:", r2)

RMSE: 15.30026440172439
R2 Score: 0.8347739170252666


**SHOW SPARK INTERNALS**

In [ ]:
print("Number of partitions:", final_ml_df.rdd.getNumPartitions())

Number of partitions: 2


In [ ]:
final_ml_df.explain(True)

== Parsed Logical Plan ==
'Project ['features, 'ARRIVAL_DELAY]
+- Project [MONTH#18, DAY#19, AIRLINE#21, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, DEPARTURE_DELAY#28, ARRIVAL_DELAY#39, AIRLINE_IDX#134, ORIGIN_IDX#181, DEST_IDX#229, UDF(struct(MONTH_double_VectorAssembler_b92d325c67f9, cast(MONTH#18 as double), DAY_double_VectorAssembler_b92d325c67f9, cast(DAY#19 as double), DEPARTURE_DELAY_double_VectorAssembler_b92d325c67f9, cast(DEPARTURE_DELAY#28 as double), AIRLINE_IDX, AIRLINE_IDX#134, ORIGIN_IDX, ORIGIN_IDX#181, DEST_IDX, DEST_IDX#229)) AS features#237]
   +- Project [MONTH#18, DAY#19, AIRLINE#21, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, DEPARTURE_DELAY#28, ARRIVAL_DELAY#39, AIRLINE_IDX#134, ORIGIN_IDX#181, UDF(cast(DESTINATION_AIRPORT#25 as string)) AS DEST_IDX#229]
      +- Filter UDF(DESTINATION_AIRPORT#25)
         +- Filter atleastnnonnulls(1, DESTINATION_AIRPORT#25)
            +- Project [MONTH#18, DAY#19, AIRLINE#21, ORIGIN_AIRPORT#24, DESTINATION_AIRPORT#25, DEPARTURE

**Basic Exploration**

In [ ]:
print("Total Number of Flights:", df.count())
df.select(
    "AIRLINE",
    "ORIGIN_AIRPORT",
    "DESTINATION_AIRPORT",
    "DEPARTURE_DELAY",
    "ARRIVAL_DELAY"
).show(5)


Total Number of Flights: 60000
+-------+--------------+-------------------+---------------+-------------+
|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DEPARTURE_DELAY|ARRIVAL_DELAY|
+-------+--------------+-------------------+---------------+-------------+
|     AS|           ANC|                SEA|            -11|          -22|
|     AA|           LAX|                PBI|             -8|           -9|
|     US|           SFO|                CLT|             -2|            5|
|     AA|           LAX|                MIA|             -5|           -9|
|     AS|           SEA|                ANC|             -1|          -21|
+-------+--------------+-------------------+---------------+-------------+
only showing top 5 rows



**Average Delay by Airline**

In [ ]:
df_clean = df.dropna(subset=["DEPARTURE_DELAY", "ARRIVAL_DELAY"])
print("Records After Cleaning:", df_clean.count())

Records After Cleaning: 58263


In [ ]:
from pyspark.sql.functions import col

avg_delay_airline = df_clean.groupBy("AIRLINE") \
    .avg("DEPARTURE_DELAY") \
    .withColumnRenamed("avg(DEPARTURE_DELAY)", "AVG_DELAY") \
    .orderBy(col("AVG_DELAY").desc())

avg_delay_airline.show(10)

+-------+------------------+
|AIRLINE|         AVG_DELAY|
+-------+------------------+
|     F9|17.681818181818183|
|     NK|16.978146853146853|
|     MQ|15.377777777777778|
|     AA|14.096903460837886|
|     UA| 13.42800711040885|
|     OO|11.277795891750896|
|     WN|10.498047361861238|
|     EV| 7.477967246327874|
|     B6| 6.625472021970477|
|     HA| 6.187274909963985|
+-------+------------------+
only showing top 10 rows



**Avg of Most Delayed Airports**

In [ ]:
airport_delay = df_clean.groupBy("ORIGIN_AIRPORT") \
    .avg("DEPARTURE_DELAY") \
    .withColumnRenamed("avg(DEPARTURE_DELAY)", "AVG_DELAY") \
    .orderBy(col("AVG_DELAY").desc())

airport_delay.show(10)

+--------------+------------------+
|ORIGIN_AIRPORT|         AVG_DELAY|
+--------------+------------------+
|           ILG|              91.0|
|           LAW| 68.28571428571429|
|           GUC| 67.64285714285714|
|           ESC|              61.5|
|           CLL|             53.15|
|           ILM|              50.4|
|           ASE|38.383720930232556|
|           SAF| 37.94444444444444|
|           SPS| 37.57142857142857|
|           CMX|37.142857142857146|
+--------------+------------------+
only showing top 10 rows



**Monthly Delay Trend**

In [ ]:
monthly_delay = df_clean.groupBy("MONTH") \
    .avg("DEPARTURE_DELAY") \
    .withColumnRenamed("avg(DEPARTURE_DELAY)", "AVG_DELAY") \
    .orderBy("MONTH")

monthly_delay.show()

+-----+------------------+
|MONTH|         AVG_DELAY|
+-----+------------------+
|    1| 8.695694756172209|
|    2|15.303565287171844|
|    3|3.7962264150943397|
+-----+------------------+



**Cancelled Flights Analysis**

In [ ]:
df.groupBy("CANCELLED").count().show()

+---------+-----+
|CANCELLED|count|
+---------+-----+
|        1| 1644|
|        0|58356|
+---------+-----+



**Top 10 Airlines by Number of Flights**

In [ ]:
df.groupBy("AIRLINE") \
  .count() \
  .orderBy(col("count").desc()) \
  .show(10)

+-------+-----+
|AIRLINE|count|
+-------+-----+
|     WN|12160|
|     DL| 8053|
|     OO| 6292|
|     EV| 6161|
|     AA| 5870|
|     UA| 5096|
|     US| 4302|
|     MQ| 3793|
|     B6| 2936|
|     AS| 1781|
+-------+-----+
only showing top 10 rows



In [ ]:
pdf_airline = avg_delay_airline.limit(10).toPandas()
pdf_monthly = monthly_delay.toPandas()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

**Top 10 Airlines by Average Departure Delay**

In [ ]:
fig = px.bar(
    pdf_airline,
    x='AIRLINE',
    y='AVG_DELAY',
    title='Top 10 Airlines by Average Departure Delay',
    labels={'AIRLINE':'Airline', 'AVG_DELAY':'Average Delay (Minutes)'},
    color='AVG_DELAY',
    color_continuous_scale='Blues'
)
fig.show()

**Monthly Average Departure Delay (Interactive Line Chart)**

In [ ]:
fig = px.line(
    pdf_monthly,
    x='MONTH',
    y='AVG_DELAY',
    title='Monthly Average Flight Delays',
    labels={'MONTH':'Month', 'AVG_DELAY':'Average Delay (Minutes)'},
    markers=True
)
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

**Cancelled vs Non-Cancelled Flights (Interactive Pie Chart)**

In [ ]:
cancel_df = df.groupBy("CANCELLED").count().toPandas()

fig = px.pie(
    cancel_df,
    values='count',
    names='CANCELLED',
    title='Cancelled vs Non-Cancelled Flights',
    color='CANCELLED',
    color_discrete_map={0:'green', 1:'red'}
)
fig.show()

**Departure Delay vs Arrival Delay (Interactive Scatter)**

In [ ]:
# Convert a sample of data to Pandas to keep it interactive
sample_df = df_clean.select("DEPARTURE_DELAY", "ARRIVAL_DELAY").sample(fraction=0.01, seed=42).toPandas()

fig = px.scatter(
    sample_df,
    x='DEPARTURE_DELAY',
    y='ARRIVAL_DELAY',
    title='Departure Delay vs Arrival Delay (Sample 1%)',
    labels={'DEPARTURE_DELAY':'Departure Delay (min)', 'ARRIVAL_DELAY':'Arrival Delay (min)'},
    opacity=0.7,
    trendline="ols",   # Adds regression line
    hover_data={'DEPARTURE_DELAY':True, 'ARRIVAL_DELAY':True}
)
fig.show()

**Heatmap of Average Delay by Airline & Month**

In [ ]:
# Aggregate avg delay by airline and month
airline_month = df_clean.groupBy("AIRLINE","MONTH") \
    .avg("DEPARTURE_DELAY") \
    .withColumnRenamed("avg(DEPARTURE_DELAY)", "AVG_DELAY") \
    .orderBy("AIRLINE","MONTH")

pdf_airline_month = airline_month.toPandas()

fig = px.density_heatmap(
    pdf_airline_month,
    x='MONTH',
    y='AIRLINE',
    z='AVG_DELAY',
    color_continuous_scale='Blues',
    title='Heatmap: Average Departure Delay by Airline & Month',
    labels={'MONTH':'Month','AIRLINE':'Airline','AVG_DELAY':'Avg Delay (min)'}
)
fig.show()

**Top 10 Airports by Number of Flights (Interactive Horizontal Bar)**

In [ ]:
top_airports = df.groupBy("ORIGIN_AIRPORT") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(10)

pdf_top_airports = top_airports.toPandas()

fig = px.bar(
    pdf_top_airports,
    x='count',
    y='ORIGIN_AIRPORT',
    orientation='h',
    title='Top 10 Airports by Number of Flights',
    labels={'count':'Number of Flights', 'ORIGIN_AIRPORT':'Airport'},
    color='count',
    color_continuous_scale='Cividis'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()